In [ ]:
import numpy as np

class robo_world():
    
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.ground = [1,3,0,2]
        self.treasure = 1
        self.robo_position = 2
        self.done = False
        return np.array([2, 1])
        
    def step(self, move):
        # 0 - left, 1 - rigth
        reward = -1
        rp = self.robo_position
        if move == 0:
            rp -= 1
        else:
            rp += 1
        
        next_state = self.ground[rp]
        
        if next_state == 2:
            self.done = True
            reward = 3
        elif next_state == 3:
            if self.treasure == 1:
                reward = 10
                self.treasure = 0
            else:
                reward = -1
        elif next_state == 1:
            self.done = True
            reward = -10
            
        self.robo_position = rp
        
        return [np.array([rp, self.treasure]), reward, self.done]
            
        
    
    
            

In [ ]:
rw = robo_world()
rw.reset()

In [ ]:
print(rw.step(1))

In [ ]:
import random

action_set = [0,1]
rw = robo_world()
for i in range(10):
    rw.reset()
    print("Trajectory: ",i)
    total_reward = 0
    for i in range(10):
        action = random.choice(action_set)
        state, reward, is_done = rw.move(action)
    #     print(position)
        total_reward += reward
        print([position,treasure, reward, is_done])
        if is_done: 
            print('total_reward', total_reward)
            break

In [ ]:
from torch.distributions import Categorical
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

gamma = 0.99

class Pi(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Pi, self).__init__()
        layers = [
        nn.Linear(in_dim, out_dim),    
#         nn.Linear(in_dim, 2),
#         nn.ReLU(),
#         nn.Linear(2, out_dim),
        ]
        self.model = nn.Sequential(*layers)
        self.onpolicy_reset()
        self.train() # установить режим обучения

    def onpolicy_reset(self):
        self.log_probs = []
        self.rewards = []

    def forward(self, x):
        pdparam = self.model(x)
        return pdparam

    def act(self, state):
        x = torch.from_numpy(state.astype(np.float32)) # преобразование в тензор
        pdparam = self.forward(x) # прямой проход
        pd = Categorical(logits=pdparam) # вероятностное распределен34 
        action = pd.sample() # pi(a|s) выбор действия по распределению pd
        log_prob = pd.log_prob(action) # логарифм вероятности pi(a|s)
        self.log_probs.append(log_prob) # сохраняем для обучения
        return action.item()

def train(pi, optimizer):
    # Внутренний цикл градиентного восхождения в алгоритме REINFORCE
    T = len(pi.rewards)
    rets = np.empty(T, dtype=np.float32) # отдачи
    future_ret = 0.0
    # эффективное вычисление отдачи
    for t in reversed(range(T)):
        future_ret = pi.rewards[t] + gamma * future_ret
        rets[t] = future_ret
    rets = torch.tensor(rets)
    log_probs = torch.stack(pi.log_probs)
    loss = - log_probs * rets # член градиента; знак минуса для максимизации
    loss = torch.sum(loss)
    optimizer.zero_grad()
    loss.backward() # обратное распространение, вычисление градиентов
    optimizer.step() # градиентное восхождение, обновление весов
    return loss

In [136]:
# env = gym.make('CartPole-v0')
env = robo_world()
in_dim = 2 #env.observation_space.shape[0] # 4
out_dim = 2 #env.action_space.n # 2
pi = Pi(in_dim, out_dim) # стратегия pi_theta для REINFORCE
optimizer = optim.Adam(pi.parameters(), lr=0.1)

loss_arr = []
rew = []
for epi in range(300):
    state = env.reset()
    for t in range(200): # 200 — максимальное количество шагов в cartpole
        action = pi.act(state)
        state, reward, done = env.step(action)
        pi.rewards.append(reward)
#         env.render()
        if done:
            break
    loss = train(pi, optimizer) # обучение в эпизоде
    loss_arr.append(loss)
    total_reward = sum(pi.rewards)
#     print(total_reward)
    rew.append(total_reward)
    solved = total_reward > 11.0
    pi.onpolicy_reset() # обучение по актуальному опыту: очистить память после обучения
#     print(f'Episode {epi}, loss: {loss}, \
#     total_reward: {total_reward}, solved: {solved}')

In [ ]:
rew

In [137]:
%matplotlib auto
import matplotlib.pyplot as plt
plt.plot(rew)

Using matplotlib backend: Qt5Agg


In [143]:
state = env.reset()
print(pi.act(np.array([1,0])))

1


In [140]:
state

array([2, 1])